In [1]:
import pandas as pd
import pyomo.environ as pyo

In [2]:
# Helper functions
def read_vehicle_cost(file_path):
    """
    This function reads the vehicle cost from the file and returns a dictionary of vehicle cost

    parameters: file_path: str: path to the file containing the vehicle cost
    """
    if file_path is None:
        raise ValueError("file_path cannot be None")
    
    data = pd.read_csv(file_path)
    vehicle_cost = {}
    for _, row in data.iterrows():
        vehicle_cost[row["ID"]] = row["Cost ($)"]

    return vehicle_cost



In [3]:
# Data

purchased_cost_data = read_vehicle_cost("dataset/vehicles.csv")

In [4]:
# Model

model = pyo.ConcreteModel()

model.years = pyo.Set(initialize = range(2023, 2039))
model.vehicle_types = pyo.Set(initialize = purchased_cost_data.keys())
model.size_buckets = pyo.Set(initialize = ["S1", "S2", "S3", "S4"])
model.distance_buckets = pyo.Set(initialize = ["D1", "D2", "D3", "D4"])
model.fuel_types = pyo.Set(initialize = ["Electricity", "B20", "LNG", "BioLNG", "HVO"])

In [5]:
model.C_buy = pyo.Param(model.vehicle_types, initialize = purchased_cost_data, within = pyo.NonNegativeReals)

In [6]:
def read_vehicle_range(file_path):
    """
    This function reads the vehicle range from the file and returns a dictionary of vehicle range

    parameters: file_path: str: path to the file containing the vehicle range
    """
    if file_path is None:
        raise ValueError("file_path cannot be None")
    
    data = pd.read_csv(file_path)
    vehicle_range = {}
    for _, row in data.iterrows():
        vehicle_range[row["ID"]] = row["Yearly range (km)"]

    return vehicle_range

In [7]:
vehicle_range_data = read_vehicle_range("dataset/vehicles.csv")

In [8]:
def read_fuel_cost(file_path):
    """
    This function reads the fuel cost from the file and returns a dictionary of fuel cost

    parameters: file_path: str: path to the file containing the fuel cost
    """
    if file_path is None:
        raise ValueError("file_path cannot be None")
    
    data = pd.read_csv(file_path)
    fuel_cost = {}
    for _, row in data.iterrows():
        key = f"{row['Fuel']}_{int(row['Year'])}"
        fuel_cost[key] = float(row["Cost ($/unit_fuel)"])

    return fuel_cost

In [9]:
fuel_cost_data = read_fuel_cost("dataset/fuels.csv")


In [10]:
model.fuel_cost = pyo.Param(model.fuel_types, model.years, initialize = lambda model, f, y: fuel_cost_data[f"{f}_{y}"], within = pyo.NonNegativeReals)

In [11]:
def read_fuel_consumption(file_path):
    """
    This function reads the fuel consumption from the file and returns a dictionary of fuel consumption

    parameters: file_path: str: path to the file containing the fuel consumption
    """
    data = pd.read_csv(file_path)
    fuel_consumption = {}
    for _, row in data.iterrows():
        fuel_consumption[row["ID"]] = row["Consumption (unit_fuel/km)"]

    return fuel_consumption

In [12]:
fuel_consumption_data = read_fuel_consumption("dataset/vehicles_fuels.csv")

In [13]:
model.fuel_consumption = pyo.Param(model.vehicle_types, initialize = fuel_consumption_data, within = pyo.NonNegativeReals)

In [14]:
model.fuel_consumption.pprint()

fuel_consumption : Size=192, Index=vehicle_types, Domain=NonNegativeReals, Default=None, Mutable=False
    Key            : Value
       BEV_S1_2023 : 0.893043
       BEV_S1_2024 : 0.893043
       BEV_S1_2025 : 0.893043
       BEV_S1_2026 : 0.893043
       BEV_S1_2027 : 0.868161
       BEV_S1_2028 : 0.868161
       BEV_S1_2029 : 0.868161
       BEV_S1_2030 : 0.868161
       BEV_S1_2031 : 0.868161
       BEV_S1_2032 : 0.868161
       BEV_S1_2033 : 0.868161
       BEV_S1_2034 : 0.868161
       BEV_S1_2035 : 0.868161
       BEV_S1_2036 : 0.868161
       BEV_S1_2037 : 0.868161
       BEV_S1_2038 : 0.868161
       BEV_S2_2023 :   0.8968
       BEV_S2_2024 :   0.8968
       BEV_S2_2025 :   0.8968
       BEV_S2_2026 :   0.8968
       BEV_S2_2027 :  0.85459
       BEV_S2_2028 :  0.85459
       BEV_S2_2029 :  0.85459
       BEV_S2_2030 :  0.85459
       BEV_S2_2031 :  0.85459
       BEV_S2_2032 :  0.85459
       BEV_S2_2033 :  0.85459
       BEV_S2_2034 :  0.85459
       BEV_S2_2035 :  0.85459


In [15]:
def read_fuel_emission_factors(file_path):
    """
    This function reads the fuel consumption from the file and returns a dictionary of fuel emissions

    parameters: file_path: str: path to the file containing the fuel consumption
    """
    data = pd.read_csv(file_path)
    emission_cost = {}
    for _,row in data.iterrows():
        key = f"{row["Fuel"]}_{int(row["Year"])}"
        emission_cost[key] = float(row["Emissions (CO2/unit_fuel)"])

    return emission_cost

In [16]:
emission_factor = read_fuel_emission_factors("dataset/fuels.csv")


In [17]:
model.emission_factor = pyo.Param(model.fuel_types, model.years, initialize = lambda model, f, y: emission_factor[f"{f}_{y}"], within = pyo.NonNegativeReals )

In [18]:
model.emission_factor.pprint()

emission_factor : Size=80, Index=fuel_types*years, Domain=NonNegativeReals, Default=None, Mutable=False
    Key                   : Value
            ('B20', 2023) :  3.04858
            ('B20', 2024) :  3.04858
            ('B20', 2025) :  3.04858
            ('B20', 2026) :  3.04858
            ('B20', 2027) :  3.04858
            ('B20', 2028) :  3.04858
            ('B20', 2029) :  3.04858
            ('B20', 2030) :  3.04858
            ('B20', 2031) :  3.04858
            ('B20', 2032) :  3.04858
            ('B20', 2033) :  3.04858
            ('B20', 2034) :  3.04858
            ('B20', 2035) :  3.04858
            ('B20', 2036) :  3.04858
            ('B20', 2037) :  3.04858
            ('B20', 2038) :  3.04858
         ('BioLNG', 2023) : 0.378439
         ('BioLNG', 2024) : 0.378439
         ('BioLNG', 2025) : 0.378439
         ('BioLNG', 2026) : 0.378439
         ('BioLNG', 2027) : 0.378439
         ('BioLNG', 2028) : 0.378439
         ('BioLNG', 2029) : 0.378439
         ('

In [19]:
def read_demand(file_path):
    """
    This function reads the demand of cars from the file and returns a dict of demand in Km

    parameters: file_path: str: path to file the containing the demand of the cars
    """
    data = pd.read_csv(file_path)
    demand = {}
    for _, row in data.iterrows():
        key = f"{int(row["Year"])}_{row["Size"]}_{row["Distance"]}"
        demand[key] = row["Demand (km)"]

    return demand 


In [20]:
demand_data = read_demand("dataset/demand.csv")

In [21]:
demand_data

{'2023_S1_D1': 869181,
 '2023_S1_D2': 2597094,
 '2023_S1_D3': 3292011,
 '2023_S1_D4': 414315,
 '2023_S2_D1': 995694,
 '2023_S2_D2': 1383196,
 '2023_S2_D3': 778008,
 '2023_S2_D4': 133677,
 '2023_S4_D1': 14576,
 '2023_S4_D2': 754717,
 '2023_S4_D3': 118899,
 '2023_S4_D4': 1809,
 '2023_S3_D1': 2183475,
 '2023_S3_D2': 2431901,
 '2023_S3_D3': 1002466,
 '2023_S3_D4': 205426,
 '2024_S1_D1': 877242,
 '2024_S1_D2': 2716195,
 '2024_S1_D3': 3336604,
 '2024_S1_D4': 419981,
 '2024_S2_D1': 1043209,
 '2024_S2_D2': 1391987,
 '2024_S2_D3': 807076,
 '2024_S2_D4': 133712,
 '2024_S4_D1': 15200,
 '2024_S4_D2': 780979,
 '2024_S4_D3': 122186,
 '2024_S4_D4': 1893,
 '2024_S3_D1': 2239440,
 '2024_S3_D2': 2453497,
 '2024_S3_D3': 1029375,
 '2024_S3_D4': 208440,
 '2025_S1_D1': 877707,
 '2025_S1_D2': 2836223,
 '2025_S1_D3': 3446113,
 '2025_S1_D4': 439767,
 '2025_S2_D1': 1082065,
 '2025_S2_D2': 1410584,
 '2025_S2_D3': 821678,
 '2025_S2_D4': 137231,
 '2025_S4_D1': 15615,
 '2025_S4_D2': 791079,
 '2025_S4_D3': 123252,
 

In [22]:
model.demand = pyo.Param(model.years, model.size_buckets, model.distance_buckets, initialize = lambda model,y, s, d: demand_data[f"{y}_{s}_{d}"], within = pyo.NonNegativeReals)

In [23]:
model.demand.pprint()

demand : Size=256, Index=years*size_buckets*distance_buckets, Domain=NonNegativeReals, Default=None, Mutable=False
    Key                : Value
    (2023, 'S1', 'D1') :  869181
    (2023, 'S1', 'D2') : 2597094
    (2023, 'S1', 'D3') : 3292011
    (2023, 'S1', 'D4') :  414315
    (2023, 'S2', 'D1') :  995694
    (2023, 'S2', 'D2') : 1383196
    (2023, 'S2', 'D3') :  778008
    (2023, 'S2', 'D4') :  133677
    (2023, 'S3', 'D1') : 2183475
    (2023, 'S3', 'D2') : 2431901
    (2023, 'S3', 'D3') : 1002466
    (2023, 'S3', 'D4') :  205426
    (2023, 'S4', 'D1') :   14576
    (2023, 'S4', 'D2') :  754717
    (2023, 'S4', 'D3') :  118899
    (2023, 'S4', 'D4') :    1809
    (2024, 'S1', 'D1') :  877242
    (2024, 'S1', 'D2') : 2716195
    (2024, 'S1', 'D3') : 3336604
    (2024, 'S1', 'D4') :  419981
    (2024, 'S2', 'D1') : 1043209
    (2024, 'S2', 'D2') : 1391987
    (2024, 'S2', 'D3') :  807076
    (2024, 'S2', 'D4') :  133712
    (2024, 'S3', 'D1') : 2239440
    (2024, 'S3', 'D2') : 2453

In [24]:
def read_carbon_emission(file_path):
    """
    This function reads the carbon limits of each years 

    parameters: file_path: str: path to the file containing the carbon emissions
    """
    data = pd.read_csv(file_path)
    carbon_emission = {}
    for _, row in data.iterrows():
        carbon_emission[row["Year"]] = row["Carbon emission CO2/kg"]

    return carbon_emission

In [25]:
carbon_limit_data = read_carbon_emission("dataset/carbon_emissions.csv")

In [26]:
model.years.pprint()

years : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :   16 : {2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038}


In [27]:
model.carbon_limit = pyo.Param(model.years, initialize = carbon_limit_data, within = pyo.NonNegativeReals)

In [28]:
def read_cost_profiles(file_path):
    """
    This function reads the resale value from and returns a dictinary

    parameter: file_path: str: path to the file containing the carbon emissions 
    """    
    if file_path is None:
        raise ValueError("file_path cannot be None")
    data = pd.read_csv(file_path)
    resale_value = {}
    insurance_cost = {}
    maintenance_cost = {}

    for _, row in data.iterrows():
        year = int(row['End of Year'])
        resale_value[year] = float(row['Resale Value %'])
        insurance_cost[year] = float(row['Insurance Cost %'])
        maintenance_cost[year] = float(row['Maintenance Cost %'])

    return resale_value, insurance_cost, maintenance_cost


In [29]:
resale_value_dict, insurance_cost_dict, maintenance_cost_dict = read_cost_profiles("dataset/cost_profiles.csv")


In [33]:
maintenance_cost_dict

{1: 1.0,
 2: 3.0,
 3: 5.0,
 4: 7.0,
 5: 9.0,
 6: 11.0,
 7: 13.0,
 8: 15.0,
 9: 17.0,
 10: 19.0}

In [30]:
model.resale_value = pyo.Param(initialize=resale_value_dict, within=pyo.NonNegativeReals)
model.insurance_cost = pyo.Param(initialize=insurance_cost_dict, within=pyo.NonNegativeReals)
model.maintenance_cost = pyo.Param(initialize=maintenance_cost_dict, within=pyo.NonNegativeReals)

ERROR: Rule failed for Param 'resale_value' with index 1: KeyError: "Cannot
treat the scalar component 'resale_value' as an indexed component"
ERROR: Constructing component 'resale_value' from data=None failed:
        KeyError: "Cannot treat the scalar component 'resale_value' as an
        indexed component"


KeyError: "Cannot treat the scalar component 'resale_value' as an indexed component"